In [1]:
%load_ext autoreload
%autoreload 2

import torch
from utils import load_module

if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [2]:
config = load_module("./configs/diffusion_hippocampus.py")

variables = dict(DATASET_PATH="X:/Datasets/Diffusion_MRI/", CHECKPOINTS_PATH="X:/Checkpoints/")
context = config.get_context(device, variables)

In [ ]:
config = load_module("./configs/qsm_deep_grey_matter.py")

variables = dict(DATASET_PATH="X:/Datasets/DGM/segmentation_3T_ps18_v3/", CHECKPOINTS_PATH="X:/Checkpoints/")
context = config.get_context(device, variables)

In [ ]:
fasd_dataset = context.dataset.get_cohort_dataset('fasd')

In [ ]:
fasd_dataset.preload_and_transform_subjects()

In [ ]:
fasd_dataset

In [3]:
untransformed_subject = context.dataset.subjects[0]
print("Original labels:")
print(untransformed_subject['whole_roi']['label_values'])

subject = context.dataset[0]
print("\nTransformed labels:")
print(subject['y']['label_values'])

inverse_subject = subject.apply_inverse_transform(warn=False)
print("\nInverse transformed labels:")
print(inverse_subject['y']['label_values'])

Original labels:
{'left_whole': 1, 'right_whole': 2}

Transformed labels:
{'left_whole': 1, 'right_whole': 1}


RuntimeError: The number of input channels was expected to be 2, but it is 1

In [ ]:
print(subject["X"])

In [ ]:
for key, value in context.dataset[1].items():
    print(key, value, type(value))
    if isinstance(value, dict):
        for key, value2 in value.items():
            print(key, value2)

In [ ]:
def sample_data(loader):
    while True:
        for batch in loader:
            yield batch
           
for i in range(2):
    loader = sample_data(context.dataloader)
    batch = next(loader)
    for key, val in batch.items():
        print(key, val.shape, val.dtype)

In [ ]:
x = torch.randn(20, 20, 20)
for y in x.split([3, 2, 5, 10]):
    print(y.shape)


In [ ]:
import wandb
wandb.init(project="test-project-2")

In [ ]:
import random

for i in range(10):
    log_dict = {
        'something': {
            foo: {'mean': random.random(), 'std': random.random()}
            for foo in "ABC"
        }
    }
    wandb.log(log_dict)

In [ ]:
import numpy as np
import pandas as pd

for i in range(10):
    evaluation = {}
    
    for structure in ("A", "B", "C"):
        columns = ["TP", "FP", "TN", "FN", 'dice', "jaccard"]
        S = 25
        subjects = [f'subject_{i:01}' for i in range(S)]

        df = pd.DataFrame(data=np.random.randint(50, 100, size=(S, 4)), columns=["TP", "FP", "TN", "FN"])
        df.insert(loc=0, column='Subject', value=subjects)

        TP, FP, TN, FN = df["TP"], df["FP"], df["TN"], df["FN"]

        df['dice'] = 2 * TP / (2 * TP + FP + FN)
        df['jaccard'] = TP / (TP + FP + FN)
        evaluation[f"Structure {structure}"] = wandb.Table(dataframe=df)
        
    wandb.log({f'Segmentation Evaluation': evaluation})

In [ ]:
wandb.finish()

In [ ]:
from data_processing import *

In [ ]:
%pip install openpyxl

In [ ]:
import pandas as pd

file_path = 'X:\Datasets\Diffusion_MRI\Attributes\demographics.xlsx'
df = pd.read_excel(file_path, index_col=0)
#subject_col = df.columns[0]
data = df.to_dict(orient='dict')

data

In [ ]:
import pathlib
print(pathlib.Path('yourPath.example').suffix) 

In [ ]:
import json

file_path = 'X:\Datasets\DGM\subject_attributes\dgm_label_names.json'
with open(file_path) as f:
    data = json.load(f)
data

In [ ]:
import pandas as pd
file_path = 'X:\Datasets\DGM\subject_attributes\dgm_label_names.csv'
df = pd.read_csv(file_path, index_col=0)
data = df.to_dict()
data

In [ ]:
from utils import is_sequence

def test_func(*args):
    if is_sequence(args) and len(args) == 1 and is_sequence(args[0]):
        args = args[0]
    print(args)
    
test_func("a", "b", "c")
test_func(("a", "b", "c"))
test_func(["a", "b", "c"])



In [ ]:
import torch
a = torch.tensor([1, 0, 1]).bool()
b = torch.tensor([1, 1, 0]).bool()


In [ ]:
import torchio as tio
import torch
from evaluators import SegmentationEvaluator
import pandas as pd

def pd_to_dict(elem):
    if isinstance(elem, dict):
        return {
            key: pd_to_wandb(val)
            for key, val in elem.items()
        }
    elif isinstance(elem, pd.DataFrame):
        return elem.to_dict()
    return elem

def pd_to_wandb(elem):
    if isinstance(elem, dict):
        return {
            key: pd_to_wandb(val)
            for key, val in elem.items()
        }
    elif isinstance(elem, pd.DataFrame):
        return wandb.Table(dataframe=elem)
    return elem

wandb.init(project="test-project-4")
for i in range(10):
    label_values = {letter: val for val, letter in enumerate("ABCDE")}

    subjects = [
        tio.Subject({
            'name': f'subject_{i:02}',
            'pred': tio.LabelMap(
                tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
                label_values=label_values
            ),
            'target': tio.LabelMap(
                tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
                label_values=label_values
            ),
        })
        for i in range(20)
    ]

    seg_evaluator = SegmentationEvaluator(
        prediction_label_name='pred', target_label_name='target', stats_to_output=['FP', 'TP', 'dice'], 
    )

    log_dict = seg_evaluator(subjects)
    wandb.log(pd_to_wandb(log_dict))
wandb.finish()

In [ ]:
label_map = tio.LabelMap(
    tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
    label_values=label_values
)

tio.OneHot()(label_map)

In [ ]:
x = torch.randn(10).unsqueeze(1)
mask = torch.tensor([0, 1, 1, 0]).bool().unsqueeze(0)
x, mask = torch.broadcast_tensors(x, mask)

print(x.shape, mask.shape)

x[mask]

In [ ]:
from evaluators import VolumeStatsEvaluator
import pandas as pd
import torchio as tio


label_values = {f'label_{letter}': (val + 1) for val, letter in enumerate("abcde")}

subjects = [
    tio.Subject({
        'name': f'subject_{i:02}',
        'label': tio.LabelMap(
            tensor=torch.randint(0, 5, size=(1, 20, 20, 20)),
            label_values=label_values
        ),
        'md': tio.ScalarImage(
            tensor=torch.randn(size=(1, 20, 20, 20)) * 2 + 1,
        ),
        'fa': tio.ScalarImage(
            tensor=torch.randn(size=(1, 20, 20, 20)) * 0.5 - 3,
        ),
    })
    for i in range(20)
]

volume_stats_eval = VolumeStatsEvaluator(
    label_map_name='label', image_names=['md', 'fa']
)

volume_stats_eval(subjects)


In [ ]:
from torch.utils.data.dataloader import default_collate

def merge_dicts(a, b):
    merged_dict = {}
    for key in a.keys():
        if key in b:
            merged_dict[key] = merge_dicts(a[key], b[key])
        else:
            merged_dict[key] = a[key]
    for key in b.keys():
        if key not in a:
            merged_dict[key] = b[key]
    return merged_dict

dict1 = {'mean': {'hippo': {'md': {'subject_1': 1}}}}
dict2 = {'mean': {'hippo': {'md': {'subject_2': 2}}}}
dict3 = {'mean': {'hippo': {'fa': {'subject_1': 3}}}}
dict4 = {'mean': {'hippo': {'fa': {'subject_2': 4}}}}

out_dict = merge_dicts(dict1, dict2)

out_dict